In [1]:
import torch

# Enable expandable segments (run before initializing tensors)
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# Monitor memory usage
print(torch.cuda.memory_summary())

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |      0 B   |      0 B   |      0 B   |      0 B   |
|       from large pool |      0 B   |      0 B   |      0 B   |      0 B   |
|       from small pool |      0 B   |      0 B   |      0 B   |      0 B   |
|---------------------------------------------------------------------------|
| Active memory         |      0 B   |      0 B   |      0 B   |      0 B   |
|       from large pool |      0 B   |      0 B   |      0 B   |

In [2]:
from pathlib import Path

model_name = 'yolo11l.pt' # The starting model weights
num_epochs = 100
image_size = 960
dataset = Path('../datasets/player_detection/')
data_yaml_path = dataset / 'data.yaml'
print(data_yaml_path.exists())
run_name = 'v_1_1large_long'
project_name = 'Pitch_Analyzer'

True


In [3]:
from ultralytics import settings
import mlflow

settings.update({"mlflow": True})
mlflow.set_experiment(project_name)
mlflow.autolog()

In [4]:
import gc
torch.cuda.empty_cache()
gc.collect()

120

# model_path = model.best  # Path to best.pt
# onnx_path = model_path.with_suffix('.onnx')
# model.export(format="onnx")abs
model = YOLO("yolo11l.pt")
dir(model)
# trainer.model.export(format='onnx')  # Export to ONNX
# mlflow.log_artifact(onnx_path)  # Log ONNX model

In [5]:
from ultralytics import YOLO

model = YOLO(model_name)

print(f"Starting training with model: {model_name}")
print(f"Dataset config: {data_yaml_path}")
print(f"Epochs: {num_epochs}, Image Size: {image_size}")
print(f"Run Name: {run_name}")

torch.cuda.empty_cache()

model = YOLO(model_name)

results = model.train(
    data=data_yaml_path,
    epochs=num_epochs,
    imgsz=image_size,
    batch=0.8, # Automatically use 80% of GPU
    project=project_name,
    name=run_name
)

torch.cuda.empty_cache()

Starting training with model: yolo11l.pt
Dataset config: ../datasets/player_detection/data.yaml
Epochs: 100, Image Size: 960
Run Name: v_1_1large_long
New https://pypi.org/project/ultralytics/8.3.108 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.107 🚀 Python-3.11.2 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce GTX 1070, 8111MiB)
engine/trainer: task=detect, mode=train, model=yolo11l.pt, data=../datasets/player_detection/data.yaml, epochs=100, time=None, patience=100, batch=0.8, imgsz=960, save=True, save_period=-1, cache=False, device=None, workers=8, project=Pitch_Analyzer, name=v_1_1large_long3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, 

train: Scanning /home/bbrzyski/studia/DFL_football/PitchAnalyzer_AI/datasets/player_detection/train/labels.cache... 505 images, 0 backgrounds, 0 corrupt: 100%|██████████| 505/505 [00:00<?, ?it/s]

AutoBatch: Computing optimal batch size for imgsz=960 at 80.0% CUDA memory utilization.
AutoBatch: CUDA:0 (NVIDIA GeForce GTX 1070) 7.92G total, 0.24G reserved, 0.23G allocated, 7.45G free
      Params      GFLOPs  GPU_mem (GB)  forward (ms) backward (ms)                   input                  output


    25313564       196.4         2.710           102         398.1        (1, 3, 960, 960)                    list
    25313564       392.8         4.201         148.1         535.2        (2, 3, 960, 960)                    list
    25313564       785.6         7.084         284.1         688.2        (4, 3, 960, 960)                    list
CUDA out of memory. Tried to allocate 50.00 MiB. GPU 0 has a total capacity of 7.92 GiB of which 71.62 MiB is free. Including non-PyTorch memory, this process has 6.85 GiB memory in use. Of the allocated memory 6.65 GiB is allocated by PyTorch, and 33.62 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)
CUDA out of memory. Tried to allocate 450.00 MiB. GPU 0 has a total capacity of 7.92 GiB of which 301.62 MiB is f

train: Scanning /home/bbrzyski/studia/DFL_football/PitchAnalyzer_AI/datasets/player_detection/train/labels.cache... 505 images, 0 backgrounds, 0 corrupt: 100%|██████████| 505/505 [00:00<?, ?it/s]
val: Scanning /home/bbrzyski/studia/DFL_football/PitchAnalyzer_AI/datasets/player_detection/valid/labels.cache... 110 images, 0 backgrounds, 0 corrupt: 100%|██████████| 110/110 [00:00<?, ?it/s]


...

    100/100       5.4G     0.5804     0.2829     0.7886         21        960: 100%|██████████| 169/169 [02:13<00:00,  1.27it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:05<00:00,  3.78it/s]

                   all        110       2626      0.876      0.785      0.818      0.624



100 epochs completed in 3.928 hours.
Optimizer stripped from Pitch_Analyzer/v_1_1large_long3/weights/last.pt, 51.2MB
Optimizer stripped from Pitch_Analyzer/v_1_1large_long3/weights/best.pt, 51.2MB

Validating Pitch_Analyzer/v_1_1large_long3/weights/best.pt...
Ultralytics 8.3.107 🚀 Python-3.11.2 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce GTX 1070, 8111MiB)
YOLO11l summary (fused): 190 layers, 25,282,396 parameters, 0 gradients, 86.6 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:05<00:00,  3.33it/s]


                   all        110       2626      0.847      0.808      0.835      0.627
                  ball         95         97      0.837      0.424      0.517      0.212
            goalkeeper         76         91      0.728      0.857      0.875      0.718
                player        110       2192      0.969      0.983      0.991      0.841
               referee        110        246      0.854      0.967      0.959      0.739
Speed: 0.4ms preprocess, 46.8ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to Pitch_Analyzer/v_1_1large_long3
MLflow: results logged to /home/bbrzyski/studia/DFL_football/PitchAnalyzer_AI/runs/mlflow
MLflow: disable with 'yolo settings mlflow=False'


import shutil
# Path where YOLO saves the weights
trained_model_path = Path(f"../runs/detect/{run_name}/weights/best.pt")
output_model_dir = Path(f"../models/player_detection/{run_name}")
# Create your custom models directory if it doesn't exist
os.makedirs(output_model_dir, exist_ok=True)

# Move the trained model
shutil.copy(trained_model_path, f"{output_model_dir}/best.pt")
print(f"Model saved to {output_model_dir}/best.pt")